In [107]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from datetime import datetime as dt
from PIL import Image
import time
import os
import re



In [ ]:
dir = r'/Users/marcinpestka/Desktop/python/'+team
os.mkdir(dir)

TypeError: can only concatenate str (not "WebElement") to str

In [126]:
path = r'./chromedriver'
driver = webdriver.Chrome(executable_path = path)
driver.get('https://www.fifa.com/fifaplus/en/tournaments/mens/worldcup/qatar2022/teams')

driver.implicitly_wait(2)
teams = driver.find_elements(By.CLASS_NAME, 'flag-with-info_flagCountry__Yw8QR') 

for team in teams:
    t = team.text
    driver2 = webdriver.Chrome(executable_path = path)
    driver2.get('https://www.fifa.com/fifaplus/en/tournaments/mens/worldcup/qatar2022/teams/'+team.text.lower()+'/squad')
    driver2.implicitly_wait(2)

    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver2.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver2.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver2.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    #getting all players 
    players = driver2.find_elements(By.CLASS_NAME, 'player-badge-card_badgeCard__2DJ4B') 
    #getting team name

    #creating dir 
    dir = r'/Users/marcinpestka/Desktop/python/'+t
    os.mkdir(dir)

    for player in players:
        #creating file name out of players name
        text = player.text
        index = [m.start() for m in re.finditer('\n', text)]
        file_name = text[:index[0]] +'_'+ text[index[0]+1:index[1]]

        # finding url of image
        test = player.find_elements(By.CLASS_NAME,"player-badge-card_playerImage__301X0")
        test = test[0].value_of_css_property("background-image")
        test = test[5:][:len(test[0])-3]

        # getting and saving the img
        img_content = requests.get(test).content
        img_file = io.BytesIO(img_content)
        image = Image.open(img_file)
        file_pth = dir+'//'+file_name+'.png'

        with open(file_pth, 'wb') as file:
            image.save(file, 'png')
    driver2.close()

# closing the driver
driver.close()

    

IndexError: list index out of range